In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import joblib
import uuid


In [ ]:
uniqueid = uuid.uuid4().int & (1 << 64)-1


In [ ]:
# https://www.kaggle.com/c/nlp-getting-started : NLP Disaster Tweets
dataset_dir = "imdb"
model_dir = "models"

df = joblib.load("../dataframes/df_imdb.pkl")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
print((df.sentiment == 1).sum()) # positive
print((df.sentiment == 0).sum()) # negative

In [ ]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df.review)

In [ ]:
len(counter)

In [ ]:
counter

In [ ]:
counter.most_common(5)

In [ ]:
num_unique_words = len(counter)

In [ ]:
# Split dataset into training and validation set
train_size = int(df.shape[0] * 0.8)

train_df = df[:train_size]
val_df = df[train_size:]

# split text and labels
train_sentences = train_df.review.to_numpy()
train_labels = train_df.sentiment.to_numpy()
val_sentences = val_df.review.to_numpy()
val_labels = val_df.sentiment.to_numpy()

In [ ]:
train_sentences.shape, val_sentences.shape

In [ ]:
# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [ ]:
# each word has unique index
word_index = tokenizer.word_index

In [ ]:
word_index

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [ ]:
print(train_sentences[10:15])
print(train_sequences[10:15])

In [ ]:
# Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
# max_length = max([len(text) for text in train_sequences])
max_length = 128
max_length

In [ ]:

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

In [ ]:
train_padded[10]

In [ ]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

In [ ]:
# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [ ]:
reverse_word_index

In [ ]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [ ]:
decoded_text = decode(train_sequences[10])

print(train_sequences[10])
print(decoded_text)

In [ ]:
from tensorflow.keras import layers


In [ ]:
# 'softmax' activation function returns a probability distribution
# Binary for 0-1, Categorical for 2 or more classes, SparseCategorical for when labels are integers
# Dropout is used to prevent overfitting by randomly setting inputs to 0 at a low rate

def LSTM_w_Loss_Binary():
    model = keras.models.Sequential()
    model.add(layers.Embedding(num_unique_words, 32,
            input_length=max_length, name="embeddingL"))
    model.add(layers.LSTM(64, dropout=0.1))
    model.add(layers.Dense(1, activation="sigmoid"))
    loss = keras.losses.BinaryCrossentropy(from_logits=False)
    optim = keras.optimizers.Adam(lr=0.001)
    metrics = ["accuracy"]
    model.compile(loss=loss, optimizer=optim, metrics=metrics)
    return model
    
def LSTM_w_Loss_Sparse_Cat():
    model = keras.models.Sequential()
    model.add(layers.Embedding(num_unique_words, 32,
            input_length=max_length, name="embeddingL"))
    model.add(layers.LSTM(64, dropout=0.1))
    model.add(layers.Dense(2, activation="softmax"))
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optim = keras.optimizers.Adam(lr=0.001)
    metrics = ["accuracy"]
    model.compile(loss=loss, optimizer=optim, metrics=metrics)
    return model


In [ ]:


model = LSTM_w_Loss_Sparse_Cat()

model.summary()

In [ ]:
train_padded.shape

In [ ]:
history = model.fit(train_padded, train_labels, epochs=4, validation_data=(val_padded, val_labels), verbose=1)

In [ ]:

# plotting training graph

plt.plot(history.history['loss'])


In [ ]:
predictions = model.predict(train_padded)


In [ ]:

predictions.shape


In [ ]:
predictions

In [ ]:

# predictions = [1 if p > 0.5 else 0 for p in predictions]
# predictions

In [ ]:

val_loss, val_acc = model.evaluate(val_padded, val_labels)
val_loss, val_acc


In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])


plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)


In [ ]:

print(train_sentences[10:20])

print(train_labels[10:20])
print(predictions[10:20])


In [ ]:
model.save(f'../{model_dir}/tensorflow_{uniqueid}_NN_model_{val_acc}')


In [ ]:

model = keras.Model(inputs=model.input,
                    outputs=[model.get_layer("embeddingL").output])

feature = model.predict(val_padded)


In [ ]:
feature

In [ ]:
feature.shape